<a href="https://colab.research.google.com/github/kangwongyu795/mob/blob/main/fake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/yoyo-nb/Thin-Plate-Spline-Motion-Model.git

Cloning into 'Thin-Plate-Spline-Motion-Model'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 115 (delta 43), reused 32 (delta 31), pack-reused 50
Receiving objects: 100% (115/115), 32.66 MiB | 28.15 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [2]:
cd Thin-Plate-Spline-Motion-Model

/content/Thin-Plate-Spline-Motion-Model


In [3]:
!mkdir checkpoints
!pip3 install wldhx.yadisk-direct
!curl -L $(yadisk-direct https://disk.yandex.com/d/QFEAZ-wXGug4KA) -o checkpoints/vox.pth.tar
# !curl -L $(yadisk-direct https://disk.yandex.com/d/vk5dirE6KNvEXQ) -o checkpoints/taichi.pth.tar
# !curl -L $(yadisk-direct https://disk.yandex.com/d/IVtro0k2MVHSvQ) -o checkpoints/mgif.pth.tar
# !curl -L $(yadisk-direct https://disk.yandex.com/d/B3ipFzpmkB1HIA) -o checkpoints/ted.pth.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  334M  100  334M    0     0  47.7M      0  0:00:07  0:00:07 --:--:-- 60.5M


In [16]:
import torch

# edit the config
device = torch.device('cuda:0')
dataset_name = 'vox' # ['vox', 'taichi', 'ted', 'mgif']
source_image_path = './assets/source.png'
driving_video_path = './assets/driving.mp4'
output_video_path = './generated.mp4'
config_path = 'config/vox-256.yaml'
checkpoint_path = 'checkpoints/vox.pth.tar'
predict_mode = 'relative' # ['standard', 'relative', 'avd']
find_best_frame = False # when use the relative mode to animate a face, use 'find_best_frame=True' can get better quality result

pixel = 256 # for vox, taichi and mgif, the resolution is 256*256
if(dataset_name == 'ted'): # for ted, the resolution is 384*384
    pixel = 384

if find_best_frame:
  !pip install face_alignment

In [17]:
try:
  import imageio
  import imageio_ffmpeg
except:
  !pip install imageio_ffmpeg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os

warnings.filterwarnings("ignore")

source_image = imageio.imread(source_image_path)
reader = imageio.get_reader(driving_video_path)

source_image = resize(source_image, (pixel, pixel))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (pixel, pixel))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani


HTML(display(source_image, driving_video).to_html5_video())

In [18]:
from demo import load_checkpoints
inpainting, kp_detector, dense_motion_network, avd_network = load_checkpoints(config_path = config_path, checkpoint_path = checkpoint_path, device = device)

In [19]:
from demo import make_animation
from skimage import img_as_ubyte

if predict_mode=='relative' and find_best_frame:
    from demo import find_best_frame as _find
    i = _find(source_image, driving_video, device.type=='cpu')
    print ("Best frame: " + str(i))
    driving_forward = driving_video[i:]
    driving_backward = driving_video[:(i+1)][::-1]
    predictions_forward = make_animation(source_image, driving_forward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions_backward = make_animation(source_image, driving_backward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions = predictions_backward[::-1] + predictions_forward[1:]
else:
    predictions = make_animation(source_image, driving_video, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)

#save resulting video
# Save resulting video
imageio.mimsave('test.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)


HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 169/169 [00:12<00:00, 14.02it/s]


In [23]:
import cv2
import imageio
from skimage import img_as_ubyte
from IPython.display import HTML

# 모자이크 처리를 위한 함수 정의
def apply_mosaic(frame, x, y, width, height, scale):
    small_frame = cv2.resize(frame[y:y+height, x:x+width], (scale, scale), interpolation=cv2.INTER_LINEAR)
    mosaic_frame = cv2.resize(small_frame, (width, height), interpolation=cv2.INTER_NEAREST)
    frame[y:y+height, x:x+width] = mosaic_frame
    return frame

# 비디오 파일 열기
video = cv2.VideoCapture('test.mp4')

# 비디오 속성 가져오기
fps = video.get(cv2.CAP_PROP_FPS)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 모자이크 처리를 위한 설정
mosaic_x = 100  # 모자이크 적용 영역의 시작 x 좌표
mosaic_y = 100  # 모자이크 적용 영역의 시작 y 좌표
mosaic_width = min(width - mosaic_x, 200)  # 모자이크 적용 영역의 가로 크기
mosaic_height = min(height - mosaic_y, 200)  # 모자이크 적용 영역의 세로 크기
mosaic_scale = 20  # 모자이크 처리에 사용할 작은 사각형의 크기

# 비디오 저장을 위한 객체 초기화
output = cv2.VideoWriter('test_mosaic.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# 비디오 프레임 단위로 처리
while True:
    # 프레임 읽기
    ret, frame = video.read()

    # 읽은 프레임이 없으면 종료
    if not ret:
        break

    # 이미지를 OpenCV 형식으로 변환
    frame_cv = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 모자이크 처리 적용
    frame_cv = apply_mosaic(frame_cv, mosaic_x, mosaic_y, mosaic_width, mosaic_height, mosaic_scale)

    # OpenCV 형식에서 다시 이미지로 변환
    frame_rgb = cv2.cvtColor(frame_cv, cv2.COLOR_RGB2BGR)

    # 모자이크 처리된 프레임 저장
    output.write(frame_rgb)

# 객체 해제
video.release()
output.release()

# 저장된 비디오 재생
HTML(f'<video controls><source src="test_mosaic.mp4" type="video/mp4"></video>')


In [37]:
import cv2
import numpy as np
from IPython.display import HTML, display
import imageio
from base64 import b64encode

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def apply_mosaic(image, x, y, width, height, scale):
    roi = image[y:y+height, x:x+width]
    small_roi = cv2.resize(roi, (scale, scale), interpolation=cv2.INTER_LINEAR)
    mosaic_roi = cv2.resize(small_roi, (width, height), interpolation=cv2.INTER_NEAREST)
    image[y:y+height, x:x+width] = mosaic_roi
    return image

# 비디오 파일 열기
video_path = 'test.mp4'
video = cv2.VideoCapture(video_path)

# 비디오 속성 가져오기
fps = video.get(cv2.CAP_PROP_FPS)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 모자이크 처리를 위한 설정
mosaic_scale = 10  # 모자이크 처리에 사용할 작은 사각형의 크기

# 비디오 저장을 위한 객체 초기화
output = cv2.VideoWriter('test_mosaic.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# 비디오 프레임 단위로 처리
while True:
    # 프레임 읽기
    ret, frame = video.read()

    # 읽은 프레임이 없으면 종료
    if not ret:
        break

    # 얼굴 검출을 위해 프레임을 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 검출을 위해 이미지를 8-bit 그레이스케일로 변환
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # 검출된 얼굴들에 대해 모자이크 처리 수행
    for (x, y, w, h) in faces:
        frame = apply_mosaic(frame, x, y, w, h, scale=mosaic_scale)

    # 모자이크 처리된 프레임 저장
    output.write(frame)

# 객체 해제
video.release()
output.release()

# 비디오 출력
output_path = 'test_mosaic.mp4'

# 출력할 비디오 파일을 base64로 인코딩
with open(output_path, 'rb') as f:
    video_data = b64encode(f.read()).decode('utf-8')

# HTML 비디오 태그 생성
html_video = f'''
<video controls>
    <source src="data:video/mp4;base64,{video_data}" type="video/mp4">
</video>
'''

# HTML로 비디오 출력
display(HTML(html_video))


In [38]:
import cv2
import numpy as np
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import imageio

# Load the mosaic result video
mosaic_result_path = 'test_mosaic.mp4'
mosaic_result_reader = imageio.get_reader(mosaic_result_path)
mosaic_result_frames = [frame for frame in mosaic_result_reader]

# Display the mosaic result video
def display_mosaic_result(frames):
    fig = plt.figure(figsize=(8, 6))
    ims = []
    for frame in frames:
        im = plt.imshow(frame, animated=True)
        plt.axis('off')
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

HTML(display_mosaic_result(mosaic_result_frames).to_html5_video())
